In [1]:
import tensorflow as tf
import cv2
from ultralytics import YOLO
from utils import handle_face_blur
import cvzone

In [2]:
converter = tf.lite.TFLiteConverter.from_saved_model('age_model_export')
tflite_model = converter.convert()

with open('age_model_export.tflite', 'wb') as f:
  f.write(tflite_model)

In [2]:
# Load TFLite Model

interpreter = tf.lite.Interpreter(model_path='age_model_export.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [3]:
def check_is_child(image, top_left_corner, bottom_right_corner):
  image = image[top_left_corner[1]:bottom_right_corner[1], top_left_corner[0]:bottom_right_corner[0]]

  IMAGE_SIZE = (100, 100)

  image = tf.image.resize(image, IMAGE_SIZE)
  image = tf.image.rgb_to_grayscale(image)
  image = tf.cast(image, tf.float32) / 255.0

  # Adicionar a dimensão de batch (de [100, 100, 1] para [1, 100, 100, 1])
  image = tf.expand_dims(image, axis=0)  # [1, 100, 100, 1]

  interpreter.set_tensor(input_details[0]['index'], image)

  interpreter.invoke()

  prediction = interpreter.get_tensor(output_details[0]['index'])

  return prediction.argmax()

In [4]:
def resize_by_width(image, new_width):
    # Obter as dimensões da imagem original
    original_height, original_width = image.shape[:2]

    # Calcular a nova altura mantendo a proporção
    scale_ratio = new_width / original_width
    new_height = int(original_height * scale_ratio)

    return new_height

In [10]:
group_age = {
  0: '0-12',
  1: '13-18',
  2: '19-35',
  3: '36-59',
  4: '60+'
}

In [31]:
model = YOLO('yolov8n-face.pt')

image_path = 'family.jpg'

image = cv2.imread(image_path)

cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Image', 1200, resize_by_width(image, 1200))
cv2.imshow('Image', image)
cv2.waitKey(0)

face_results = model(image, conf=0.3, stream=True)

for face_result in face_results:
    for box in face_result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        top_left_corner = (x1, y1)
        bottom_right_corner = (x2, y2)

        group =     (image, top_left_corner, bottom_right_corner)

        cvzone.putTextRect(image, f'Age: {group_age[group]}', (x1 - 60, y1 - 20), 5, 8)
        cv2.rectangle(image, top_left_corner, bottom_right_corner, (0, 255, 0), 3)
        
        if group == 0:
            image = handle_face_blur(image, top_left_corner, bottom_right_corner)
            

cv2.imshow('Image', image)

output_path = 'family_processed.jpg'
cv2.imwrite(output_path, image)

cv2.waitKey(0)
cv2.destroyAllWindows()



0: 448x640 4 faces, 31.9ms
Speed: 6.0ms preprocess, 31.9ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)
